<a href="https://colab.research.google.com/github/AndreaNeti/AAPP-Maze-Generator-Challenge/blob/main/mazegenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maze Generator Challenge 2022
Andrea Sanguineti - 218443

In [1]:
!apt install clang-10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
clang-10 is already the newest version (1:10.0.0-4ubuntu1).
clang-10 set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [2]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

Cloning into 'benchmark'...
remote: Enumerating objects: 7724, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 7724 (delta 15), reused 39 (delta 13), pack-reused 7671
Receiving objects: 100% (7724/7724), 2.42 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (5140/5140), done.
Cloning into 'benchmark/googletest'...
remote: Enumerating objects: 25621, done.
remote: Counting objects: 100% (319/319), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 25621 (delta 191), reused 229 (delta 159), pack-reused 25302
Receiving objects: 100% (25621/25621), 11.82 MiB | 16.77 MiB/s, done.
Resolving deltas: 100% (18879/18879), done.


In [3]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 9.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.25.1") 
-- git version: v1.7.1-25-g0ce66c00 normalized to 1.7.1.25
-- Version: 1.7.1.25
-- Looking for shm_open in rt
-- Looking for shm_open in rt - found
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL - Success
-- Performing Test HAVE_CXX_FLAG_WERROR
-- Performing Test HAVE_CXX_FLAG_WERROR - Success
-- Performing Test HAVE_CXX_FLAG_PEDANTIC
-- Per

Classes UnionFind, ForestSet and ListSet extend DisjoinedSets with different implementations

In [4]:
%%writefile disjointSets.hpp
class DisjointSets {
    public:
        virtual bool connected(int idX, int idY) = 0;
        virtual void connect(int idX, int idY) = 0;
};
// UNION BY RANK IMPLEMENTATION
class UnionRank : public DisjointSets
{
    typedef struct cell
    {
        cell *p;
        int id;
        int rank = 0;
    } cell;

private:
    cell *cells;

    cell *getCell(int id)
    {
        return &cells[id];
    }
    void link(cell *x, cell *y)
    {
        // x and y are roots
        if (x->rank > y->rank)
            y->p = x;
        else
        {
            x->p = y;
            if (x->rank == y->rank)
                y->rank++;
        }
    }
    cell *findSet(cell *x)
    {
        if (x != x->p)
            x->p = findSet(x->p);
        return x->p;
    }

public:
    UnionRank(int size)
    {
        cells = new cell[size];
        for (int i = 0; i < size; i++)
            cells[i] = {&cells[i], i}; // at start every cell is its own root
    }
    ~UnionRank()
    {
        free(cells);
        cells = NULL;
    }
    bool connected(int idX, int idY)
    {
        if (idX == idY)
            return true;
        return findSet(getCell(idX)) == findSet(getCell(idY));
    }
    void connect(int idX, int idY)
    {
        if (idX != idY)
            link(findSet(getCell(idX)), findSet(getCell(idY)));
    }
};
// FOREST IMPLEMENTATION
class ForestSet : public DisjointSets
{
    typedef struct cell
    {
        cell *p;
    } cell;

private:
    cell *cells;

    cell *getCell(int id)
    {
        return &cells[id];
    }
    cell *findSet(int idX)
    {
        cell *x = getCell(idX);
        while (x != x->p)
            x = x->p;
        return x;
    };

public:
    ForestSet(int size)
    {
        cells = new cell[size];
        for (int i = 0; i < size; i++)
            cells[i] = {&cells[i]}; // at start every cell points to itself
    }
    ~ForestSet()
    {
        free(cells);
        cells = NULL;
    }    
    bool connected(int idX, int idY)
    {
        if (idX == idY)
            return true;
        return findSet(idX) == findSet(idY);
    }
    void connect(int idX, int idY)
    {
        if (idX != idY)
            findSet(idY)->p = findSet(idX); // connect the root of Y to the root of X
    }
};

// LIST IMPLEMENTATION
class ListSet : public DisjointSets
{
    typedef struct cell
    {
        cell *head;
        cell *next;
        cell *tail;
    } cell;

private:
    cell *cells;

    cell *getCell(int id)
    {
        return &cells[id];
    }
    cell *findSet(int idX)
    {
        return getCell(idX)->head;
    }

public:
    ListSet(int size)
    {
        cells = new cell[size];
        for (int i = 0; i < size; i++)
            cells[i] = {&cells[i], NULL, &cells[i]}; // at start every cell points to itself
    }
    ~ListSet()
    {
        free(cells);
        cells = NULL;
    }    
    bool connected(int idX, int idY)
    {
        if (idX == idY)
            return true;
        return findSet(idX) == findSet(idY);
    }
    void connect(int idX, int idY)
    {
        if (idX == idY)
            return;
        cell *xHead = findSet(idX);
        cell *yHead = findSet(idY);
        if (xHead != yHead)
        {
            // append Y after X
            xHead->tail->next = yHead;
            // update with the new tail
            xHead->tail = yHead->tail;
            // set the head of X to all Y nodes
            yHead->head = xHead;
            cell *y = yHead->next;
            // change the head in the Y set nodes
            while (y != NULL)
            {
                y->head = xHead;
                y = y->next;
            }
        }
    }
};

Writing disjointSets.hpp


The Maze class generates a maze using disjoined sets, one for each cell at start.<br><br>
During the generation, a random wall is selected and if the two adjacent cells are in different sets, the two sets are merged and the wall destroyed, otherwise is kept and no more selected.<br><br>

In [11]:
%%writefile maze.hpp
#pragma once

#include "disjointSets.hpp"

#define UP 0
#define RIGHT 1
#define DOWN 2
#define LEFT 3

template <typename T, typename std::enable_if<std::is_base_of<DisjointSets, T>::value>::type * = nullptr>
class Maze
{
    typedef struct wall
    {
        int idC1;
        int idC2; // the two cells Id having this wall in common
    } wall;

private:
    int rows, cols;
    int numWalls;
    T *sets;
    wall *walls;
    bool **openings;
    wall *generateWalls()
    {
        wall *walls = new wall[numWalls];
        int r, c, w = 0, selectedCell;
        // generate vertical walls
        for (r = 0, selectedCell = 0; r < this->rows - 1; r++)
        {
            for (c = 0; c < this->cols - 1; c++, selectedCell++)
            {
                walls[w++] = {selectedCell, selectedCell + 1};          // vertical wall
                walls[w++] = {selectedCell, selectedCell + this->cols}; // horizontal wall
            }
            walls[w++] = {selectedCell, selectedCell + this->cols}; // last column has no cell to the right
            selectedCell++;
        }
        for (c = 0; c < this->cols - 1; c++, selectedCell++) // last row has no cells below
            walls[w] = {selectedCell, selectedCell + 1};
        // randomly shuffle the walls
        std::random_shuffle(&walls[0], &walls[numWalls]);
        return walls;
    };
    bool depthFirst(int curId, int endId, int prevId)
    {
        if (curId == endId)
        {
            std::cout << "\n"
                      << curId;
            return true;
        }
        else
        {
            if ((openings[curId][RIGHT] && prevId != curId + 1 && depthFirst(curId + 1, endId, curId)) ||
                (openings[curId][DOWN] && prevId != curId + this->cols && depthFirst(curId + this->cols, endId, curId)) ||
                (openings[curId][LEFT] && prevId != curId - 1 && depthFirst(curId - 1, endId, curId)) ||
                (openings[curId][UP] && prevId != curId - this->cols && depthFirst(curId - this->cols, endId, curId)))
            {
                std::cout << " -> " << curId;
                return true;
            }
            else
                return false;
        }
    };

public:
    Maze(int rows, int cols)
    {
        srand(time(NULL));
        this->rows = rows;
        this->cols = cols;
        int numCells = rows * cols;
        numWalls = 2 * numCells - (rows + cols);
        sets = new T(numCells);
        walls = generateWalls();
        openings = new bool *[numCells];
        for (int i = 0; i < numCells; i++)
            openings[i] = new bool[4]{};
    };
    ~Maze()
    {
        delete (sets);
        sets = NULL;
        delete[] walls;
        walls = NULL;
        for (int i = 0; i < 4; i++)
            delete[] openings[i];
        delete[] openings;
        openings = NULL;
    }
    void generateMaze(int startId, int endId)
    {
        int i = 0;
        while (!sets->connected(startId, endId) && i < numWalls)
        {
            wall *selectedWall = &walls[i];
            int idX = selectedWall->idC1;
            int idY = selectedWall->idC2;
            if (!sets->connected(idX, idY)) // destroy the wall
            {
                if (idY == idX + 1 && cols > 1)
                { // vertical wall
                    openings[idX][RIGHT] = true;
                    openings[idY][LEFT] = true;
                }
                else
                { //  horizontal wall
                    openings[idX][DOWN] = true;
                    openings[idY][UP] = true;
                }
                sets->connect(idX, idY);
            }
            i++;
        }
    }
    void printMaze()
    {
        int i, selectedCell;
        std::cout << "\n ";
        for (i = 0; i < 2 * this->cols - 1; i++)
            std::cout << "_";
        for (selectedCell = 0, i = 0; i < this->rows - 1; i++, selectedCell++)
        {
            std::cout << "\n|";
            for (int j = 0; j < this->cols - 1; j++, selectedCell++)
            {
                if (openings[selectedCell][RIGHT])
                {
                    if (openings[selectedCell][DOWN])
                        std::cout << "  ";
                    else
                        std::cout << "__";
                }
                else if (openings[selectedCell][DOWN])
                    std::cout << " |";
                else
                    std::cout << "_|";
            }
            if (openings[selectedCell][DOWN])
                std::cout << " |";
            else
                std::cout << "_|";
        }
        std::cout << "\n|";
        for (i = 0; i < this->cols - 1; i++, selectedCell++)
        {
            if (openings[selectedCell][RIGHT])
                std::cout << "__";
            else
                std::cout << "_|";
        }
        std::cout << "_|";
    };
    bool testPath(int startId, int endId)
    {
        // search the path starting from the end so the prints will be in the right order
        return depthFirst(endId, startId, endId);
    }
};

Overwriting maze.hpp


In [12]:
%%writefile test.cpp
#include <iostream>
#include "maze.hpp"
#include <benchmark/benchmark.h>

static void generateUnionRankMaze(benchmark::State& state){  
    int size = state.range(0);
    for (auto _ :state){        
        Maze<UnionRank>(size, size).generateMaze(0, size*size -1);
    }
    state.SetComplexityN(state.range(0));
}
static void generateForestSetMaze(benchmark::State& state){  
    int size = state.range(0);   
    for (auto _ :state){        
        Maze<ForestSet>(size, size).generateMaze(0, size*size -1);
    }
    state.SetComplexityN(state.range(0));
}
static void generateListSetMaze(benchmark::State& state){  
    int size = state.range(0);
    for (auto _ :state){        
        Maze<ListSet>(size, size).generateMaze(0, size*size -1);
    }
    state.SetComplexityN(state.range(0));
}
BENCHMARK(generateUnionRankMaze)
    ->RangeMultiplier(2)
    ->Range(1<<1, 1<<8)
    ->Complexity();
BENCHMARK(generateForestSetMaze)
    ->RangeMultiplier(2)
    ->Range(1<<1, 1<<8)
    ->Complexity();
BENCHMARK(generateListSetMaze)
    ->RangeMultiplier(2)
    ->Range(1<<1, 1<<8)
    ->Complexity();

BENCHMARK_MAIN();

Overwriting test.cpp


In [7]:
%%writefile main.cpp
#include <iostream>
#include "maze.hpp"

int main(int argc, char *argv[])
{
    int rows = 6;
    int cols = 8;
    if (argc >= 3)
    {
        try
        {
            rows = std ::stoi(argv[1]);
            cols = std ::stoi(argv[2]);
        }
        catch (...)
        {
            std::cout << "Error in parsing maze size\n";
        }
    }
    int startId = 0;
    int endId = (rows * cols) - 1;
    Maze maze = Maze<UnionRank>(rows, cols);
    maze.generateMaze(startId, endId);
    maze.printMaze();
    maze.testPath(startId, endId);
    return 0;
}

Writing main.cpp


In [13]:
!clang++-10 test.cpp -O3 -std=c++2a -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o test -g

As we can see Union Find implementation is the one with lowest complexity
We know that a findSet in a Union-Find data structure is O(α(n)), in the worst case we have to check/destroy ≈ 2*n^2 walls -> ≈ 4*n^2 calls to findSet(x), also a lower bound can be found and it's by destroying n^2+n walls, still in order of n^2.<br>
Hence we expect that the general complexity will be in order of O(α(n)*n^2) (A findSet repeated n^2 times).

In [19]:
!./test

2023-02-09T21:50:47+00:00
Running ./test
Run on (2 X 2200 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.37, 0.47, 0.37
--------------------------------------------------------------------
Benchmark                          Time             CPU   Iterations
--------------------------------------------------------------------
generateUnionRankMaze/2         1957 ns         1940 ns       362838
generateUnionRankMaze/4         3615 ns         3601 ns       207795
generateUnionRankMaze/8        10338 ns        10260 ns        69739
generateUnionRankMaze/16       39600 ns        39467 ns        17238
generateUnionRankMaze/32      163812 ns       163398 ns         4355
generateUnionRankMaze/64      725945 ns       721493 ns          996
generateUnionRankMaze/128    3270780 ns      3261796 ns          219
generateUnionRankMaze/256   16094629 ns     16038839 ns           46
generateUnionRankMaze_Bi

In [14]:
!clang++-10 main.cpp -O2 -std=c++2a -o main

Maze print and depth first search for path from start *(upper-left corner, ID = 0)* to end *(lower-rigth corner, ID = 47)*

Cell IDs are disposed as shown below

 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 
---|---|---|---|---|---|---|---
8  | 9 | 10 | 11 | 12 | 13 | 14 | 15
16 | 17 | 18 | 19 | 20 | 21 | 22 | 23
24 | 25 | 26 | 27 | 28 | 29 | 30 | 31
32 | 33 | 34 | 35 | 36 | 37 | 38 | 39
40 | 41 | 42 | 43 | 44 | 45 | 46 | 47

In [18]:
!./main 6 8


 _______________
| | |__ |__ | | |
| |____  _|  __ |
|  __   | | | | |
|  _| |  __  _|_|
|   | |   |   | |
|_|_|_|_|_|_|___|
0 -> 8 -> 16 -> 17 -> 18 -> 19 -> 27 -> 28 -> 29 -> 37 -> 38 -> 46 -> 47